In [2]:
pip install inscriptis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [inscriptis]
Note: you may need to restart the kernel to use updated packages.


In [3]:
import urllib.request
import pandas as pd
import urllib.error

from inscriptis import get_text

In [4]:
df = pd.read_csv("financial_news_data.csv")

In [5]:
df.duplicated().sum()

np.int64(0)

In [6]:
df.shape

(2086, 7)

In [7]:
df.isna().sum()

source            0
author          243
url               0
description     288
title             0
content           0
published_at      0
dtype: int64

In [8]:
df.head()

,source,author,url,description,title,content,published_at
0,"{'id': 'business-insider', 'name': 'Business I...",Kristine Villarroel,https://www.businessinsider.com/vintage-photos...,The market crash of 1929 triggered the Great D...,20 vintage photos show the 1929 stock market c...,The widespread panic following the market cras...,2025-05-09 09:19:01
1,"{'id': None, 'name': 'Yahoo Entertainment'}",editorial-team@simplywallst.com (Simply Wall St),https://finance.yahoo.com/news/three-stocks-ma...,The United States market has shown positive mo...,Three Stocks That May Be Trading Below Their E...,The United States market has shown positive mo...,2025-05-09 07:37:55
2,"{'id': None, 'name': 'Fresno Bee'}",Joshua Tehee,https://www.fresnobee.com/news/local/article30...,"So far in 2025, 21 restaurants, markets and ot...",Event center in Fresno closed due to unsanitar...,Reality Check is a Fresno Bee series holding t...,2025-05-09 15:30:00
3,"{'id': 'business-insider', 'name': 'Business I...",nredmond@insider.com (Nora Redmond),https://www.businessinsider.com/hsbc-loan-to-h...,"Since Trump announced his ""Liberation Day"" tar...",HSBC launched a loan specifically to help busi...,Banking giant HSBC has launched a new financin...,2025-05-09 07:38:01
4,"{'id': 'business-insider', 'name': 'Business I...",Grace Eliza Goodwin,https://www.businessinsider.com/white-house-sa...,The White House says Trump's 10% baseline tari...,The White House says Trump's baseline 10% tari...,President Donald Trump wants to keep his basel...,2025-05-09 15:50:23


In [11]:
# function to scrape data from a givne url
def get_whole_content(url, timeout=20):
    try:
        response = requests.get(url, timeout=timeout, headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        response.raise_for_status()
        return get_text(response.text)
    except requests.exceptions.Timeout:
        print(Exception(f"Timeout after {timeout} seconds"))
    except requests.exceptions.RequestException as e:
        print(Exception(f"Request failed: {str(e)}")

In [14]:
import urllib.request
import urllib.error
from urllib.parse import urlparse
import socket
import time

def get_whole_content(url, timeout=10):
    """
    Scrape content from URL with timeout and error handling
    """
    try:
        # Validate URL
        parsed = urlparse(url)
        if not parsed.scheme or not parsed.netloc:
            raise ValueError(f"Invalid URL: {url}")
            
        # Create request with headers to avoid bot detection
        req = urllib.request.Request(
            url,
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
        )
        
        # Open URL with timeout
        with urllib.request.urlopen(req, timeout=timeout) as response:
            html = response.read().decode('utf8', errors='ignore')
            text = get_text(html)
            return text
            
    except urllib.error.HTTPError as e:
        print(f"HTTP Error {e.code}: {e.reason}")
        raise
    except urllib.error.URLError as e:
        print(f"URL Error: {e.reason}")
        raise
    except socket.timeout:
        print(f"Timeout after {timeout} seconds")
        raise
    except Exception as e:
        print(f"Unexpected error: {type(e).__name__}: {str(e)}")
        raise

# Initialize lists
faulty_indexes = []
successful_count = 0

# Iterate with better error handling
for index, row in df.iterrows():
    try:
        print(f"Processing index {index}: {row['url'][:50]}...")  # Show partial URL
        
        # Add timeout to prevent hanging
        content = get_whole_content(row['url'], timeout=10)
        
        # Update the dataframe properly
        df.at[index, 'content'] = content
        
        successful_count += 1
        print(f"✓ Successfully processed index {index}")
        
        # Add delay to be respectful to servers
        time.sleep(0.5)
        
    except Exception as e:
        print(f"✗ Failed at index {index}: {str(e)}")
        faulty_indexes.append({
            'index': index,
            'url': row['url'],
            'error': str(e)
        })
        
        # Continue to next URL instead of getting stuck
        continue

print(f"\nCompleted: {successful_count} successful, {len(faulty_indexes)} failed")

Processing index 0: https://www.businessinsider.com/vintage-photos-192...
✓ Successfully processed index 0
Processing index 1: https://finance.yahoo.com/news/three-stocks-may-tr...
✓ Successfully processed index 1
Processing index 2: https://www.fresnobee.com/news/local/article306000...
Timeout after 10 seconds
✗ Failed at index 2: The read operation timed out
Processing index 3: https://www.businessinsider.com/hsbc-loan-to-help-...
✓ Successfully processed index 3
Processing index 4: https://www.businessinsider.com/white-house-says-t...
✓ Successfully processed index 4
Processing index 5: https://qz.com/btc-eth-bitcoin-price-ethereum-trum...
✓ Successfully processed index 5
Processing index 6: https://gizmodo.com/what-are-you-panic-buying-befo...
✓ Successfully processed index 6
Processing index 7: https://www.forbes.com/sites/jjkinahan/2025/05/09/...
✓ Successfully processed index 7
Processing index 8: https://www.macrumors.com/2025/05/09/apple-acquire...
✓ Successfully processed ind

index  1193
1194
index  1195
1196
1197
1198
1199
index  1200
index  1201
1202
index  1203
1204
index  1205
index  1206
1207
index  1208
1209
1210
index  1211
index  1212
1213
1214
index  1215
1216
1217
1218
1219
1220
1221
1222
1223
index  1224
1225
1226
1227
1228
1229
1230
index  1231
1232
1233
index  1234
1235
1236
1237
1238
index  1239
1240
1241
1242
index  1243
1244
1245
index  1246
index  1247
1248
index  1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
index  1261
1262
1263
1264
index  1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
index  1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
index  1295
index  1296
1297
index  1298
1299
1300
1301
index  1302
1303
index  1304
1305
1306
1307
index  1308
index  1309
index  1310
index  1311
index  1312
index  1313
1314
index  1315
1316
1317
1318
1319
index  1320
1321
1322
1323
index  1324
1325
1326
index  1327
index  1328
index  1329
index  1330
index  1331
1332
index  1333
i

In [15]:
len(faulty_indexes)

234

In [16]:
# as we have 234 indexes whose content we wer not able to scrape, we will drop these indexes

In [28]:
print(faulty_indexes[:2])

[{'index': 2, 'url': 'https://www.fresnobee.com/news/local/article306000211.html', 'error': 'The read operation timed out'}, {'index': 17, 'url': 'https://comecruisewith.com/cruise-excursion-leads-to-illness-princess-cruises-limits-drinks', 'error': 'HTTP Error 403: Forbidden'}]


In [24]:

# new_df = df.drop(index=[value for key,value in record.items() if key == "index" for record in faulty_indexs])
new_df = df.drop(index=[value for record in faulty_indexes for key, value in record.items() if key == "index"])


In [25]:
new_df.head()

,source,author,url,description,title,content,published_at
0,"{'id': 'business-insider', 'name': 'Business I...",Kristine Villarroel,https://www.businessinsider.com/vintage-photos...,The market crash of 1929 triggered the Great D...,20 vintage photos show the 1929 stock market c...,Subscribe Newsletters\n Search\n Busines...,2025-05-09 09:19:01
1,"{'id': None, 'name': 'Yahoo Entertainment'}",editorial-team@simplywallst.com (Simply Wall St),https://finance.yahoo.com/news/three-stocks-ma...,The United States market has shown positive mo...,Three Stocks That May Be Trading Below Their E...,News\n\n ...,2025-05-09 07:37:55
3,"{'id': 'business-insider', 'name': 'Business I...",nredmond@insider.com (Nora Redmond),https://www.businessinsider.com/hsbc-loan-to-h...,"Since Trump announced his ""Liberation Day"" tar...",HSBC launched a loan specifically to help busi...,Subscribe Newsletters\n Search\n Busines...,2025-05-09 07:38:01
4,"{'id': 'business-insider', 'name': 'Business I...",Grace Eliza Goodwin,https://www.businessinsider.com/white-house-sa...,The White House says Trump's 10% baseline tari...,The White House says Trump's baseline 10% tari...,Subscribe Newsletters\n Search\n Busines...,2025-05-09 15:50:23
5,"{'id': None, 'name': 'Quartz India'}",Catherine Baab,https://qz.com/btc-eth-bitcoin-price-ethereum-...,Winning the White House was supposed to transf...,"Bitcoin tops $100,000 and Ethereum surges in b...",Search\n ...,2025-05-09 12:57:00


In [26]:
new_df.shape

(1852, 7)

We successfully extracted content of 1852 articles, now we can say that our dataset is ready.

In [27]:
new_df.to_csv("financial_news_complete_content.csv", index=False)